# Build System


### Set the build parameters

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'

### Connect to the Hardware

Using the parameters set above, set up and connect to the hardware. This is robust to allow for different ChipWhisperer boards and/or targets.

In [2]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")

INFO: Found ChipWhisperer😍


In [3]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None

In [4]:
import time
time.sleep(0.05)
scope.default_setup()
scope.adc.samples = 30000
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

In [5]:
print(scope)

ChipWhisperer Nano Device
fw_version = 
    major = 0
    minor = 64
    debug = 0
io = 
    tio1         = None
    tio2         = None
    tio3         = None
    tio4         = high_z
    pdid         = True
    pdic         = False
    nrst         = True
    clkout       = 7500000.0
    cdc_settings = bytearray(b'\x01\x00\x00\x00')
adc = 
    clk_src  = int
    clk_freq = 7500000.0
    samples  = 30000
glitch = 
    repeat     = 0
    ext_offset = 0



### Building the Firmware

Each time `statemachine-base.c` is changed rerun this code.

In [42]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../firmware/simpleserial-base
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2


SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-base.c ...
arm-none-eabi-gcc -c -o objdir-CWNANO/simpleserial-base.o -mcpu=cortex-m0 -I. -DNO_EXTRA_OPT -mthumb -mfloat-abi=soft -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_1_1 -DSTM32F030x6 -DSTM32F0 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f0_nano -DPLATFORM=CWNANO -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_2_1=3 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfields -fshort-enums -Wall -Wstrict-prototypes -Wa,-adhlns=objdir-CWNANO/simpleserial-base.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/s

arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib: linker input file unused because linking not done
arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib/glibc/libc_nonshared.a: linker input file unused because linking not done


-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
arm-none-eabi-gcc -c -o objdir-CWNANO/simpleserial.o -mcpu=cortex-m0 -I. -DNO_EXTRA_OPT -mthumb -mfloat-abi=soft -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_1_1 -DSTM32F030x6 -DSTM32F0 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f0_nano -DPLATFORM=CWNANO -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_2_1=3 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfields -fshort-enums -Wall -Wstrict-prototypes -Wa,-adhlns=objdir-CWNANO/simpleserial.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy -std=gnu99 -Wno-discarded-qualifiers -Wno-unused-function -Wno-unused-variable -Wno-strict-prototypes -Wno-missing-prototypes -Wno-pointer-sign -Wno-unused-value -Wno-implicit-function-declaration  -MMD -MP -MF .dep/simpleserial.o.d .././simpleserial/simpleserial.c  -O -DNDEBUG -fPIC -fvisibility=hidden -isystem -L /

arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib: linker input file unused because linking not done
arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib/glibc/libc_nonshared.a: linker input file unused because linking not done


-e Done!
.
Compiling:
-en     .././hal/stm32f0_nano/stm32f0_hal_nano.c ...
arm-none-eabi-gcc -c -o objdir-CWNANO/stm32f0_hal_nano.o -mcpu=cortex-m0 -I. -DNO_EXTRA_OPT -mthumb -mfloat-abi=soft -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_1_1 -DSTM32F030x6 -DSTM32F0 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f0_nano -DPLATFORM=CWNANO -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_2_1=3 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfields -fshort-enums -Wall -Wstrict-prototypes -Wa,-adhlns=objdir-CWNANO/stm32f0_hal_nano.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy -std=gnu99 -Wno-discarded-qualifiers -Wno-unused-function -Wno-unused-variable -Wno-strict-prototypes -Wno-missing-prototypes -Wno-pointer-sign -Wno-unused-value -Wno-implicit-function-declaration  -MMD -MP -MF .dep/stm32f0_hal_nano.o.d .././hal/stm32f0_nano/stm32f0_hal_nano.c  -O -DNDEBUG -fPIC -fvis

...

-e Done!
.
Compiling:
-en     .././hal/stm32f0/stm32f0_hal_lowlevel.c ...
arm-none-eabi-gcc -c -o objdir-CWNANO/stm32f0_hal_lowlevel.o -mcpu=cortex-m0 -I. -DNO_EXTRA_OPT -mthumb -mfloat-abi=soft -ffunction-sections -gdwarf-2 -DSS_VER=SS_VER_1_1 -DSTM32F030x6 -DSTM32F0 -DSTM32 -DDEBUG -DHAL_TYPE=HAL_stm32f0_nano -DPLATFORM=CWNANO -DF_CPU=7372800UL -DSS_VER_2_0=2 -DSS_VER_2_1=3 -DSS_VER_1_1=1 -DSS_VER_1_0=0 -Os -funsigned-char -funsigned-bitfields -fshort-enums -Wall -Wstrict-prototypes -Wa,-adhlns=objdir-CWNANO/stm32f0_hal_lowlevel.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy -std=gnu99 -Wno-discarded-qualifiers -Wno-unused-function -Wno-unused-variable -Wno-strict-prototypes -Wno-missing-prototypes -Wno-pointer-sign -Wno-unused-value -Wno-implicit-function-declaration  -MMD -MP -MF .dep/stm32f0_hal_lowlevel.o.d .././hal/stm32f0/stm32f0_hal_lowlevel.c  -O -DNDEBUG -

arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib: linker input file unused because linking not done
arm-none-eabi-gcc: warning: /home/beth-c132/.elan/toolchains/leanprover--lean4---nightly-2023-06-20/lib/glibc/libc_nonshared.a: linker input file unused because linking not done


-e Done!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy .././hal/stm32f0/stm32f0_startup.S -o objdir-CWNANO/stm32f0_startup.o
.
LINKING:
-en     simpleserial-base-CWNANO.elf ...


/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: objdir-CWNANO/simpleserial-base.o: in function `get_pt':
/home/beth-c132/SCL/chipwhisperer-minimal/firmware/simpleserial-base/simpleserial-base.c:111: undefined reference to `lean_initialize_runtime_module'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: /home/beth-c132/SCL/chipwhisperer-minimal/firmware/simpleserial-base/simpleserial-base.c:112: undefined reference to `lean_set_panic_messages'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: /home/beth-c132/SCL/chipwhisperer-minimal/firmware/simpleserial-base/simpleserial-base.c:113: undefined reference to `initialize_Main'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: /home/beth-c132/SCL/chipwhisperer-minimal/firmware/simpleserial-base/simpleserial-base.c:114: undefined reference to `lean_set_panic_messages'
/usr/lib/gcc/arm-none-eabi/10.3.1/../../../arm-none-eabi/bin/ld: /home/beth-c132/SCL/chipwhisperer-minimal/firmw

CalledProcessError: Command 'b'cd ../firmware/simpleserial-base\nmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2\n'' returned non-zero exit status 2.

### Upload Firmware

In [7]:
cw.program_target(scope, prog, "../firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 7743 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 7743 bytes


### Define the Capture trace function

In [8]:
trace_array = []
for _ in range(100):
    msg = bytearray([0] + [0]*15)
    scope.arm()
    target.simpleserial_write('p', msg)
    scope.capture()
    # print(scope.get_last_trace())
    trace = scope.get_last_trace()
    target.simpleserial_write('x', bytearray([0]*16))
    trace_array.append(trace)
print("All done! :)")

All done! :)
